In [ ]:
from sbcbinaryformat import Streamer, Writer
import numpy as np

from GetEvent import GetEvent
import matplotlib as mpl
import matplotlib.pyplot as plt

from PIL import Image

In [ ]:
plt.rc('font', size=14)

In [ ]:
TEST_RUN = "/exp/e961/data/SBC-25-daqdata/20250718_11/"
TEST_EVT = 0

In [ ]:
data = GetEvent(TEST_RUN, TEST_EVT, "cam", strictMode=False)

In [ ]:
data

In [ ]:
plt.imshow(data["cam"]["c1"]["frame0"])

In [ ]:
plt.imshow(data["cam"]["c2"]["frame0"])

In [ ]:
plt.imshow(data["cam"]["c1"]["frame1"] - data["cam"]["c1"]["frame2"])

In [ ]:
plt.imshow(data["cam"]["c2"]["frame1"] - data["cam"]["c2"]["frame2"])

In [ ]:
image = data["cam"]["c1"]["frame1"] - data["cam"]["c1"]["frame2"]

In [ ]:
fft_image = np.fft.fft2(image, axes=(0, 1))
# shifted_fft_image = np.fft.fftshift(fft_image)
psd2D = (np.abs(fft_image)**2).mean(axis=-1)

In [ ]:
toshow = np.log10(psd2D)
# toshow = psd2D
plt.imshow(toshow, vmin=9, vmax=11, origin="lower")
cbar = plt.colorbar()
cbar.set_label("$\\log_{10}$ Noise Power", rotation=-90, labelpad=17)
plt.xlabel("Pixel Frequency X")
plt.ylabel("Pixel Frequency Y")

In [ ]:
def radial_noise_power(image):
    # Take FFT
    fft_image = np.fft.fft2(image, axes=(0, 1))
    # Take norm, average over colors
    psd2D = (np.abs(fft_image)**2).mean(axis=-1)

    # Get the frequencies
    freq_rows = np.fft.fftfreq(psd2D.shape[0])
    freq_cols = np.fft.fftfreq(psd2D.shape[1])
    freqXs, freqYs = np.meshgrid(freq_rows, freq_cols, indexing="ij")
    freqRs = np.sqrt(freqXs**2 + freqYs**2)
    rbins = freq_cols
    rs = (rbins[:-1] + rbins[1:]) / 2

    # Only include the lower half to avoid redundancy
    which_im = np.tril(freqRs) != 0

    # Set the radial power spectrum
    psdR = np.zeros(rbins.size-1)
    for irr, (rlo, rhi) in enumerate(zip(rbins[:-1], rbins[1:])):
        which_f = which_im & (freqRs >= rlo) & (freqRs < rhi)
        psdR[irr] = psd2D[which_f].mean()

    return rs, psdR

In [ ]:
freqs, psdR = radial_noise_power(image)

In [ ]:
plt.plot(freqs, np.log10(psdR))
plt.xlabel("Radial Pixel Frequency [pix$^{-1}$]")
plt.ylabel("$\\log_{10}$ Radial Power")

In [ ]:
psdRs = []
for i in range(9):
    image = data["cam"]["c1"]["frame%i" % i] - data["cam"]["c1"]["frame%i" % (i+1)]
    freqs, psdR = radial_noise_power(image)
    psdRs.append(psdR)

In [ ]:
for psdR in psdRs:
    plt.plot(freqs, np.log10(psdR))
plt.title("Camera 1 Run %s Event %i" % (TEST_RUN.split("/")[-2], TEST_EVT))
plt.xlabel("Radial Pixel Frequency [pix$^{-1}$]")
plt.ylabel("$\\log_{10}$ Radial Power")

In [ ]:
# Load a simulated bubble to overlay

In [ ]:
# denoised version
bkg_img = np.array(Image.open("/exp/e961/data/users/gputnam/renders/default_scene_denoised_S1.png").convert("RGB"))
bub_img = np.array(Image.open("/exp/e961/data/users/gputnam/renders/bubble_X-3_93_Y-4_24_Z-9_37_denoised_S1.png").convert("RGB"))


In [ ]:
# # normal version
# bkg_img = np.array(Image.open("/exp/e961/data/users/gputnam/renders/default_scene_S2.png").convert("RGB"))
# bub_img = np.array(Image.open("/exp/e961/data/users/gputnam/renders/5mm_bubble_4k.png").convert("RGB"))

In [ ]:
plt.imshow(bkg_img)

In [ ]:
plt.imshow(bub_img)

In [ ]:
bubble = bub_img - bkg_img

In [ ]:
plt.imshow(bubble)

In [ ]:
plt.imshow(data["cam"]["c1"]["frame1"] + bubble)

In [ ]:
for psdR in psdRs:
    plt.plot(freqs, np.log10(psdR), color="gray")

_, bubR = radial_noise_power(bubble)
plt.plot(freqs, np.log10(bubR), color="red", label="Simulated Bubble", linewidth=2)

plt.legend()
    
plt.xlabel("Radial Pixel Frequency [pix$^{-1}$]")
plt.ylabel("$\\log_{10}$ Radial Power")
plt.xscale("log")

In [ ]:
image = data["cam"]["c1"]["frame1"] + bubble
freq_image = np.fft.fft2(image, axes=(0, 1))
for i in range(3):
    freq_image[:, :, i][freqRs > 1e-1] = 0 
    # freq_image[:, :, i][freqRs > 5e-2] = 0 

image_denoised = np.abs(np.fft.ifft2(freq_image, axes=(0, 1)))
image_denoised = image_denoised - image_denoised.min()
image_denoised = 255*image_denoised/image_denoised.max()
image_denoised = image_denoised.astype(int)

In [ ]:
plt.imshow(image)

In [ ]:
plt.imshow(image_denoised)